In [ ]:
using Revise
using Pkg
TAMBO_PATH = "/Users/jlazar/research/TAMBO-MC/Tambo/"
Pkg.activate(TAMBO_PATH)
using Tambo
using StatsBase
using Distributions
using JLD2
using Glob

include("../figures/paperstyle.jl")

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

ALTMIN = 1.8925255158436627units.km
ALTMAX = 4.092525515843662units.km

EDGES = 10 .^ LinRange(14.5, 18, 15)
CENTS = (EDGES[1:end-1] + EDGES[2:end]) / 2
WIDTHS = EDGES[2:end] - EDGES[1:end-1];

# First, I want to make sure that our new simulation is compatible with our old simulation

In [ ]:
# We will use this
HISTS = zeros((2, 3, 2, length(CENTS))); # new/old x inj/dia/sdc x weight/variance

function is_in_zenith_range(event::Tambo.InjectionEvent, θmin=0.0, θmax=π/2) :: Bool
    return θmin < event.initial_state.direction.θ && event.initial_state.direction.θ < θmax
end

## Let's first compute this for injection level

### First we compute the rate as a funciton of energy for the new simulation

In [ ]:
sim_fs = glob("00000_00???/GraphNet_00000_00???.jld2", "/Users/jlazar/Downloads/GraphNet_00000/")

HISTS[1, 1, :, :] .= 0

for (idx, sim_f) in enumerate(sim_fs)
    
    # Load and configure
    sim = jldopen(sim_f)
    config = SimulationConfig(; filter(x->x[1]!=:geo_spline_path, sim["config"])...)
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
    
    # Maks events in proper zenith range
    zenith_mask = is_in_zenith_range.(sim["injected_events"])
    events = sim["injected_events"][zenith_mask]
    
    # Calculate rates
    # fluxes = ones(size(events["initial_state"]["energy"]))
    fluxes = Φ.(events["initial_state"]["energy"])
    wgts = Tambo.oneweight.(
        events,
        Ref(injector),
        Ref(injector.xs),
        # Ref(injector.powerlaw),
        # Ref(injector.anglesampler),
        # Ref(injector.injectionvolume),
        # Ref(geo)
    ) / (config.n * length(sim_fs))
#     rates = weights .* fluxes
    
    # Histogram events
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(wgts),
        EDGES
    )
    
    HISTS[1, 1, 1, :] += h1.weights
    
#     h2 = fit(
#         Histogram,
#         events["initial_state"]["energy"],
#         Weights(rates .^ 2),
#         EDGES
#     )
#     HISTS[1, 1, 2, :] += h2.weights
    
    close(sim)
    
end

In [ ]:
HISTS[1, 1, 1, :] ./ units.m^2 ./ WIDTHS

In [ ]:
plt = plot(
    xscale=:log10,
    yscale=:log10
)

m = HISTS[1, 1, 1, :] .> 0

plot!(
    plt,
    CENTS[m] / units.GeV,
    HISTS[1, 1, 1, m] ./ units.m^2 ./ WIDTHS[m]
)
display(plt)

### Now we compute it again for the old simulation

In [ ]:
sim_fs = ["/Users/jlazar/Downloads/Oct16th2023_WhitePaper_300k.jld2"]

HISTS[2, 1, : ,:] .= 0

for (idx, sim_f) in enumerate(sim_fs)
    sim = jldopen(sim_f)
    
    config = SimulationConfig(; filter(x->x[1]!=:geo_spline_path, sim["config"])...)
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
    
    zenith_mask = is_in_zenith_range.(sim["injected_events"])

    
    events = sim["injected_events"][zenith_mask]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    weights = Tambo.oneweight.(
        events,
        Ref(injector),
        Ref(injector.xs),
        # Ref(injector.xs),
        # Ref(injector.powerlaw),
        # Ref(injector.anglesampler),
        # Ref(injector.injectionvolume),
        # Ref(geo)
    ) / (config.n * length(sim_fs))
    rates = weights .* fluxes
    
    ## For all events ##
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(rates),
        EDGES
    )
    HISTS[2, 1, 1, :] += h1.weights
    
    h2 = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(rates .^ 2),
        EDGES
    )
    HISTS[2, 1, 2, :] += h2.weights
    
    close(sim)
    
end

### And now we make the plot

A few things to point out:
1. The bands surrounding each line are the $5\sigma$ uncertainties, _i.e._ we have pretty good knowledge of our rates.
1. As expected the uncertainty is constant with energy for the $\gamma=1$ case and grows with energy for the $\gamma=2$ case.
1. It does not seem to be falling off like $E^{-2.37}$. I will investigate this later...

In [ ]:
plt = plot(
    xscale=:log10,
    yscale=:log10,
    # yaxis=(:log10, [:auto, :auto]),
    xlabel=L"E_{\nu}~\left[\mathrm{eV}\right]",
    bottommargin=3mm,
    leftmargin=3mm,
    xlimits=(3e5, 1e9),
    size=(600, 400)
)

for idx in 1:2

    label = idx==1 ? "New" : "Old"

    rate = HISTS[idx, 1, 1, :] * units.second * 10^7.5
    err = 5 * sqrt.(HISTS[idx, 1, 2, :]) * units.second * 10^7.5
    m = rate .> 0

    plot!(
        plt,
        CENTS[m] ./ units.GeV,
        rate[m],
        label="Injection",
        ribbon=err[m],
        linetype=:stepmid
    )
end

display(plt)

### Let's find out if these are equal, _i.e._ if there difference is consistent with 0

It seems that they are close to 0, but there are still definitely some systematic differences...

In [ ]:
plt = plot(
    xscale=:log10,
    xlabel=L"E_{\nu}~\left[\mathrm{eV}\right]",
    ylabel="New-Old",
    bottommargin=3mm,
    leftmargin=3mm,
    xlimits=(3e5, 1e9),
    size=(600, 400)
)

rate1 = HISTS[1, 1, 1, :] * units.second * 10^7.5
err1 = sqrt.(HISTS[1, 1, 2, :]) * units.second * 10^7.5
rate2 = HISTS[2, 1, 1, :] * units.second * 10^7.5
err2 = sqrt.(HISTS[2, 1, 2, :]) * units.second * 10^7.5
m = rate1 .> 0 .&& rate2 .> 0

plot!(
    plt,
    CENTS[m] ./ units.GeV,
    rate1[m] - rate2[m],
    label="Injection",
    ribbon=sqrt.(err1[m] .^2 + err2[m] .^2),
    linetype=:stepmid
)

display(plt)

## Now we will run the same procedure, but checking only particle which decayed in air, and which we run CORSIKA on

In [ ]:
sim_fs = glob("00000_00???/GraphNet_00000_00???.jld2", "/Users/jlazar/Downloads/GraphNet_00000/")

HISTS[1, 2:3, :, :] .= 0

for (idx, sim_f) in enumerate(sim_fs)
    
    # Load and configure
    sim = jldopen(sim_f)
    config = SimulationConfig(; filter(x->x[1]!=:geo_spline_path, sim["config"])...)
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
    
    # Mask proper zenith range
    zenith_mask = is_in_zenith_range.(sim["injected_events"])
    events = sim["injected_events"][zenith_mask]
    
    # Calculate decay in air and should do CORSIKA masks
    dia_mask = .~Tambo.inside.(sim["proposal_events"][zenith_mask]["propped_state"]["position"], Ref(geo))
    sdc_mask = Tambo.should_do_corsika.(sim["proposal_events"][zenith_mask], Ref(plane), Ref(geo))
    
    # Calculate rates
    fluxes = Φ.(events["initial_state"]["energy"])
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / (config.n * length(sim_fs))
    
    rates = weights .* fluxes
    
    # Histogram events which decay in air
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"][dia_mask],
        Weights(rates[dia_mask]),
        EDGES
    )
    HISTS[1, 2, 1, :] += h1.weights
    
    h2 = fit(
        Histogram,
        events["initial_state"]["energy"][dia_mask],
        Weights(rates[dia_mask] .^ 2),
        EDGES
    )
    HISTS[1, 2, 2, :] += h2.weights
    
    # Histogram events require CORSIKA
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"][sdc_mask],
        Weights(rates[sdc_mask]),
        EDGES
    )
    HISTS[1, 3, 1, :] += h1.weights
    
    h2 = fit(
        Histogram,
        events["initial_state"]["energy"][sdc_mask],
        Weights(rates[sdc_mask] .^ 2),
        EDGES
    )
    HISTS[1, 3, 2, :] += h2.weights
    
    close(sim)
end

In [ ]:
sim_fs = ["/Users/jlazar/Downloads/Oct16th2023_WhitePaper_300k.jld2"]

HISTS[2, 2:3, :, :] .= 0

for (idx, sim_f) in enumerate(sim_fs)
    
    # Load and configure
    sim = jldopen(sim_f)
    config = SimulationConfig(; filter(x->x[1]!=:geo_spline_path, sim["config"])...)
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
    
    # Mask proper zenith range
    zenith_mask = is_in_zenith_range.(sim["injected_events"])
    events = sim["injected_events"][zenith_mask]
    
    # Calculate decay in air and should do CORSIKA masks
    dia_mask = .~Tambo.inside.(sim["proposal_events"][zenith_mask]["propped_state"]["position"], Ref(geo))
    sdc_mask = Tambo.should_do_corsika.(sim["proposal_events"][zenith_mask], Ref(plane), Ref(geo))
    
    # Calculate rates
    fluxes = Φ.(events["initial_state"]["energy"])
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / (config.n * length(sim_fs))
    
    rates = weights .* fluxes
    
    # Histogram events which decay in air
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"][dia_mask],
        Weights(rates[dia_mask]),
        EDGES
    )
    HISTS[2, 2, 1, :] += h1.weights
    
    h2 = fit(
        Histogram,
        events["initial_state"]["energy"][dia_mask],
        Weights(rates[dia_mask] .^ 2),
        EDGES
    )
    HISTS[2, 2, 2, :] += h2.weights
    
    # Histogram events require CORSIKA
    h1 = fit(
        Histogram,
        events["initial_state"]["energy"][sdc_mask],
        Weights(rates[sdc_mask]),
        EDGES
    )
    HISTS[2, 3, 1, :] += h1.weights
    
    h2 = fit(
        Histogram,
        events["initial_state"]["energy"][sdc_mask],
        Weights(rates[sdc_mask] .^ 2),
        EDGES
    )
    HISTS[2, 3, 2, :] += h2.weights
    
    close(sim)

end

In [ ]:
for jdx in 2:3
    
    plt = plot(
        xscale=:log10,
        # yscale=:log10,
        yaxis=(:log10, [2e-3, 1e-1]),
        xlabel=L"E_{\nu}~\left[\mathrm{eV}\right]",
        bottommargin=3mm,
        leftmargin=3mm,
        xlimits=(3e5, 1e9),
        size=(600, 400)
    )
    
    for idx in 1:2

        label = idx==1 ? "New" : "Old"

        rate = HISTS[idx, jdx, 1, :] * units.second * 10^7.5
        err = 5 * sqrt.(HISTS[idx, jdx, 2, :]) * units.second * 10^7.5
        m = rate .> 0

        plot!(
            plt,
            CENTS[m] ./ units.GeV,
            rate[m],
            label=label,
            ribbon=err[m],
            linetype=:stepmid
        )
    end
    display(plt)
end



### Let's once again look at the difference of the rates

It looks like there is potentially some systematic behavior, but these look close enough. From here on out, I am going to consider only the new simulation

In [ ]:
plt = plot(
    xscale=:log10,
    xlabel=L"E_{\nu}~\left[\mathrm{eV}\right]",
    ylabel="New-Old",
    bottommargin=3mm,
    leftmargin=3mm,
    xlimits=(3e5, 1e9),
    ylimits=(-0.004, 0.004),
    size=(600, 400)
)

plot!(
    plt,
    [CENTS[1], CENTS[end]] / units.GeV,
    [0, 0],
    linestyle=:dash,
    color=:black, 
    lw=1.5
)

labels = ["", "DIA", "SDC"]

for idx in 2:3

    label = labels[idx]
    
    rate1 = HISTS[1, idx, 1, :] * units.second * 10^7.5
    err1 = sqrt.(HISTS[1, idx, 2, :]) * units.second * 10^7.5
    rate2 = HISTS[2, idx, 1, :] * units.second * 10^7.5
    err2 = sqrt.(HISTS[2, idx, 2, :]) * units.second * 10^7.5
    m = rate1 .> 0 .&& rate2 .> 0
    
    plot!(
        plt,
        CENTS[m] ./ units.GeV,
        rate1[m] - rate2[m],
        label=label,
        ribbon=sqrt.(err1[m] .^2 + err2[m] .^2),
        linetype=:stepmid
    )
end

display(plt)

# Now I want to look at the injection spectrum a bit more closely.

By eye, it does not seem that the spectrum is following the expected $E^{-2.37}$. Looking at it a bit more closely here, it is pretty clear that this is not falling off like $E^{-2.37}$, but rather something like $E^{-1.7}$. Somehow we are getting an extra power of 0.66 or so. I guess we expect a factor of 0.33 or so from the cross section.

In [ ]:
plt = plot(
    xscale=:log10,
    yscale=:log10,
    # yaxis=(:log10, [:auto, :auto]),
    xlabel=L"E_{\nu}~\left[\mathrm{GeV}\right]",
    ylabel=L"\frac{\mathrm{d}N_{\nu}}{\mathrm{d}E_{\nu}}~\left[\mathrm{GeV}^{-1}\right]",
    bottommargin=3mm,
    leftmargin=3mm,
    xlimits=(3e5, 1e9),
    size=(600, 400)
)

rate = HISTS[1, 1, 1, :] * units.second * 10^7.5
err = 5 * sqrt.(HISTS[1, 1, 2, :]) * units.second * 10^7.5
m = rate .> 0

for γ′ in 1.5:0.1:2.0

    plot!(
        plt,
        CENTS / units.GeV,
        CENTS .^ -γ′ * (HISTS[1,1,1,5] * units.second * 10^7.5/WIDTHS[5]*units.GeV / CENTS[5]^-γ′),
        label=L"E^{-%$(γ′)}"
    )
end

plot!(
    plt,
    CENTS[m] ./ units.GeV,
    rate[m] ./ (WIDTHS[m] / units.GeV),
    label="Injection",
    linetype=:stepmid,
    grid=true,
    lw=3,
    color=:black
)

display(plt)

# Sandbox

In [ ]:
jldf = jldopen(sim_fs[1])

In [ ]:
events = jldf["injected_events"]

In [ ]:
function impact_parameter(p, d)
    r = norm_(p)
    return r * sqrt(1 - (sum(p .* d) / r)^2)
end

function impact_parameter(event)
    p = event.entry_state.position 
    d = event.entry_state.direction.proj
    return impact_parameter(p, d)
end

In [ ]:
bs = impact_parameter.(events) ./ units.km
h = fit(Histogram, bs)


plt = plot(h.edges[1][1:end-2], h.weights)
display(plt)

In [ ]:
ev.final_state.position ./ units.km

In [ ]:
(ev.final_state.position - ev.entry_state.position) ./ ev.final_state.direction.proj

In [ ]:
sim = jldopen(sim_fs[1])
config = SimulationConfig(; filter(x->x[1]!=:geo_spline_path, sim["config"])...)
geo = Tambo.Geometry(config)
plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
injector = Tambo.Injector(config)

# Mask proper zenith range
# zenith_mask = is_in_zenith_range.(sim["injected_events"])
events = sim["injected_events"]

# Calculate decay in air and should do CORSIKA masks
# dia_mask = .~Tambo.inside.(sim["proposal_events"][zenith_mask]["propped_state"]["position"], Ref(geo))
# sdc_mask = Tambo.should_do_corsika.(sim["proposal_events"][zenith_mask], Ref(plane), Ref(geo))

# Calculate rates
fluxes = Φ.(events["initial_state"]["energy"])
weights = Tambo.oneweight.(
    events,
    Ref(injector.xs),
    Ref(injector.xs),
    Ref(injector.powerlaw),
    Ref(injector.anglesampler),
    Ref(injector.injectionvolume),
    Ref(geo)
)

In [ ]:
weights

In [ ]:
jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events.jld2")

In [ ]:
jldf["config1"]

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

for idx in 1:2
    config = jldf["config$(idx)"]
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
    
    # @show config
    
    events = jldf["events$(idx)"]
    
    # mask =. units.PeV .< events["initial_state"]["energy"] .&& 
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    @show sum(rates)
    @show √(sum(rates .^ 2))
end

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)
        
    events = jldf["injection_events$(idx)"]
    
    # mask =. units.PeV .< events["initial_state"]["energy"] .&& 
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    rates = weights .* fluxes * units.second * 10^7.5
    @show sum(rates)
    @show √(sum(rates .^ 2))
end

close(jldf)

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)

    mask = .~Tambo.inside.(jldf["proposal_events$(idx)"]["propped_state"]["position"], Ref(geo))
    
    events = jldf["injection_events$(idx)"][mask]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    @show sum(rates)
    @show √(sum(rates .^ 2))
end

close(jldf)

In [ ]:
(0.6092861046390421 - 0.5856487232555672) / √(0.020122981644587333^2 + 0.034624810399983406^2)

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)

    mask = Tambo.should_do_corsika.(jldf["proposal_events$(idx)"], Ref(plane), Ref(geo))
    
    events = jldf["injection_events$(idx)"][mask]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    @show sum(rates)
    @show √(sum(rates .^ 2))
end

close(jldf)

In [ ]:
(0.24129219291299478 - 0.23949863872500352) / √(0.012974998861612498^2 + 0.020305556997366664^2)

In [ ]:
EDGES = 10 .^ (log(10, 3e14):0.25:18)
CENTS = (EDGES[1:end-1] + EDGES[2:end]) / 2
WIDTHS = EDGES[2:end] - EDGES[1:end-1]

## Now we're gonna make some plots

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

plt = plot(xscale=:log10, yscale=:log10)

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)

    # mask = .~Tambo.inside.(jldf["proposal_events$(idx)"]["propped_state"]["position"], Ref(geo))
    
    events = jldf["injection_events$(idx)"]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    
    h = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(weights .* fluxes),
        EDGES
    )
    
    plot!(
        plt, CENTS / units.GeV,
        h.weights ./ WIDTHS * units.second * units.GeV,
        linetype=:stepmid
    )
    
    @show sum(rates)
    @show √(sum(rates .^ 2))
end
display(plt)
close(jldf)

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

plt = plot(xscale=:log10)

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)

    mask = .~Tambo.inside.(jldf["proposal_events$(idx)"]["propped_state"]["position"], Ref(geo))
    
    events = jldf["injection_events$(idx)"][mask]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    
    h = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(weights .* fluxes),
        EDGES
    )
    
    plot!(plt, CENTS / units.GeV, h.weights / WIDTHS * CENTS * units.second, linetype=:stepmid)
    
    @show sum(rates)
    @show √(sum(rates .^ 2))
end
display(plt)
close(jldf)

In [ ]:
γ = 2.37
norm = 1.44e-18 / units.GeV / units.cm^2 / units.second * (1 /(100units.TeV))^-γ
Φ = Tambo.PowerLaw(γ, 1000units.GeV, 1e9units.GeV, norm)

jldf = jldopen("/Users/jlazar/Downloads/tambo_weighting_test_events3.jld2")

plt = plot(xscale=:log10)

for idx in 1:2
    config = SimulationConfig(;jldf["config$(idx)"]...)
    
    @show config
    
    config.geo_spline_path = "../resources/tambo_spline.jld2"
    geo = Tambo.Geometry(config)
    plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
    injector = Tambo.Injector(config)

    mask = Tambo.should_do_corsika.(jldf["proposal_events$(idx)"], Ref(plane), Ref(geo))
    
    events = jldf["injection_events$(idx)"][mask]
    
    fluxes = Φ.(events["initial_state"]["energy"])
    
    weights = Tambo.oneweight.(
        events,
        Ref(injector.xs),
        Ref(injector.xs),
        Ref(injector.powerlaw),
        Ref(injector.anglesampler),
        Ref(injector.injectionvolume),
        Ref(geo)
    ) / config.n
    
    rates = weights .* fluxes * units.second * 10^7.5
    
    h = fit(
        Histogram,
        events["initial_state"]["energy"],
        Weights(weights .* fluxes),
        EDGES
    )
    
    plot!(plt, CENTS / units.GeV, h.weights / WIDTHS * CENTS * units.second, linetype=:stepmid)
    
    @show sum(rates)
    @show √(sum(rates .^ 2))
end
display(plt)
close(jldf)